- We loose structure of dialogue using the tokenizer in preprocess_data

- Emotions encoded using MultiLabelBinarizer doesn't tell us
  anymore how many times a single emotion is present in the dialogue
  and where it is

In [1]:
!pip install torch==1.13.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers==4.30.0
!pip install datasets==2.13.2
!pip install accelerate -U
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 380.5 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
# system packages
from pathlib import Path
import shutil
import urllib
import tarfile
import sys
import os
# data and numerical management packages
import pandas as pd
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
# useful during debugging (progress bars)
from tqdm import tqdm
from transformers import set_seed

seed = 852

random.seed(seed)
np.random.seed(seed)
set_seed(seed)

In [3]:
dataset_folder = Path.cwd().joinpath("MELD_train_efr.json")
#dataset_path = dataset_folder.joinpath('/MELD_train_efr.json')
df = pd.read_json(dataset_folder)
#df['triggers'] = df['triggers'].fillna(value=0, inplace=False)#.replace('None', 0.0)

In [4]:
df

,episode,speakers,emotions,utterances,triggers
0,utterance_0,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 1.0, 0.0]"
1,utterance_1,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
2,utterance_2,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,utterance_3,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,utterance_4,"[Joey, Rachel, Joey, Rachel]","[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...
3995,utterance_3995,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3996,utterance_3996,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3997,utterance_3997,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3998,utterance_3998,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [5]:
triggers = df['triggers']
for row in range(triggers.shape[0]):
    for trigger in range(len(triggers[row])):
        if triggers[row][trigger] == None:
            triggers[row][trigger] = 0.0

df['triggers'] = triggers

In [5]:
emotions = df['emotions'].explode().unique()
emotions

array(['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust',
       'anger'], dtype=object)

In [ ]:
triggers = df['triggers'].explode().unique()
triggers

In [6]:
dialogues = df['utterances']
#print(sentences)
max_len_dialogue = 0
index = 0
for idx, dialogue in enumerate(dialogues):
  if len(dialogue) > max_len_dialogue:
    max_len_dialogue = len(dialogue)
    index = idx
max_len_dialogue,index

(24, 219)

In [7]:
dialogues = df['utterances']
#print(sentences)
max_len_sentence = 0
index = 0
for idx, dialogue in enumerate(dialogues):
  for idx_sentence, utterance in enumerate(dialogue):
    if len(utterance.split()) > max_len_sentence:
      max_len_sentence = len(utterance.split())
      index = idx,idx_sentence
max_len_sentence,index

(69, (1675, 11))

### Be careful for the token used during padding

In [9]:
padded_dialogues = [seq + ['[PAD]'] * (max_len_dialogue - len(seq)) for seq in dialogues]

In [10]:
padded_emotions = [seq + ['[PAD]'] * (max_len_dialogue - len(seq)) for seq in df['emotions']]

In [ ]:
triggers = df['triggers']
padded_triggers = [seq + [0] * (max_len_dialogue - len(seq)) for seq in triggers]

In [ ]:
"""tokenized_input_ids = []
tokenized_attention_mask = []
tokenized_token_type_ids = []

for utterance_list in padded_dialogues:
  utterance_input_ids = []
  utterance_attention_mask = []
  utterance_token_type_ids = []
  for utterance in utterance_list:
    tokenized_texts = tokenizer(utterance,return_tensors='pt')# padding='max_length', max_length=max_len, )
    utterance_input_ids.extend(tokenized_texts.input_ids)
    utterance_attention_mask.extend(tokenized_texts.attention_mask)
    utterance_token_type_ids.extend(tokenized_texts.token_type_ids)

  tokenized_input_ids.append(utterance_input_ids)
  tokenized_attention_mask.append(utterance_attention_mask)
  tokenized_token_type_ids.append(utterance_token_type_ids)
"""

In [ ]:
"""emotions_input_ids = []
emotions_attention_mask = []
emotions_token_type_ids = []

for utterance_list in padded_emotions:
  emotion_input_ids = []
  emotion_attention_mask = []
  emotion_token_type_ids = []
  for utterance in utterance_list:
    tokenized_texts = tokenizer(utterance,return_tensors='pt')# padding='max_length', max_length=max_len, )
    emotion_input_ids.append(tokenized_texts.input_ids)
    emotion_attention_mask.append(tokenized_texts.attention_mask)
    emotion_token_type_ids.append(tokenized_texts.token_type_ids)

  emotions_input_ids.append(emotion_input_ids)
  emotions_attention_mask.append(emotion_attention_mask)
  emotions_token_type_ids.append(emotion_token_type_ids)"""

In [ ]:
df['emotions'][:5]

In [ ]:
"""
pad = torch.from_numpy(np.array([0]))
padded_utterances_input_ids = [seq + [pad] * (max_len - len(seq)) for seq in tokenized_input_ids]
padded_utterances_attention_mask = [seq + [torch.from_numpy(np.array([1]))] * (max_len - len(seq)) for seq in tokenized_attention_mask]
padded_utterances_token_type_ids = [seq + [pad] * (max_len - len(seq)) for seq in tokenized_token_type_ids]
pad_as_1 = torch.from_numpy(np.array([1]))
padded_emotions_input_ids = [seq + [pad] * (max_len - len(seq)) for seq in emotions_input_ids]
padded_emotions_attention_mask = [seq + [pad_as_1] * (max_len - len(seq)) for seq in emotions_attention_mask]
padded_emotions_token_type_ids = [seq + [pad] * (max_len - len(seq)) for seq in emotions_token_type_ids]
"""

In [ ]:
#emotion_mapping = {emotion: idx+1 for idx, emotion in enumerate(emotions)}
#emotions_numerical_format = [[emotion_mapping[emotion] for emotion in dialogue] for dialogue in df['emotions']]
#padded_emotions = [seq + [0] * (max_len_dialogue - len(seq)) for seq in emotions_numerical_format]

In [11]:
if np.isin('[PAD]', emotions).any() == False:
  emotions = np.append(emotions,np.array(['[PAD]']))

In [26]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
model_card = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_card)
model = AutoModelForSequenceClassification.from_pretrained(model_card,
                                                           num_labels= len(emotions),#max_len_dialogue +1 padding
                                                           id2label=id_to_emotion,
                                                           label2id=emotion_to_id,
                                                           problem_type="multi_label_classification")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [13]:
from sklearn.preprocessing import LabelBinarizer
sorted_emotions = sorted(emotions)  #sort the array because Binarizer will automatically do that for one hot encoding
label_binarizer = LabelBinarizer()
label_binarizer.fit(sorted_emotions)
numerical_representation = [label_binarizer.transform(sublist) for sublist in padded_emotions]

In [9]:
# Create a dictionary from ID to emotions
id_to_emotion = {idx: emotion for idx, emotion in enumerate(emotions)}

# Create a dictionary from emotion to ID
emotion_to_id = {emotion: idx for idx, emotion in enumerate(emotions)}

# Print the dictionaries
print("ID to Emotion:", id_to_emotion)
print("Emotion to ID:", emotion_to_id)

exitmotions_numerical_format = [[emotion_to_id[emotion] for emotion in dialogue] for dialogue in padded_emotions]

ID to Emotion: {0: 'neutral', 1: 'surprise', 2: 'fear', 3: 'sadness', 4: 'joy', 5: 'disgust', 6: 'anger'}
Emotion to ID: {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger': 6}


In [60]:
exitmotions_numerical_format[0]

[0, 0, 0, 0, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]

In [15]:
data = {
    'episode' : df['episode'],
    'labels' : exitmotions_numerical_format,#emotions_input_ids, #tokenized emotions
    'emotions' : padded_emotions,#emotions_input_ids, #tokenized emotions
    'utterances': padded_dialogues,
}
df_tokenized = pd.DataFrame(data)
from sklearn.model_selection import train_test_split
train_data, temp_data = train_test_split(df_tokenized, train_size=0.8, shuffle=False)
val_data, test_data = train_test_split(temp_data, test_size=0.5, shuffle=False)
train_data.shape, val_data.shape,test_data.shape

NameError: name 'exitmotions_numerical_format' is not defined

In [ ]:
from datasets import Dataset
training_hf = Dataset.from_pandas(train_data)
validation_hf = Dataset.from_pandas(val_data)
test_hf = Dataset.from_pandas(test_data)

In [64]:
labels_np = np.array(exitmotions_numerical_format)
print((labels_np[3600]))

[0 0 1 0 0 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


In [30]:
test_emotions = df['emotions']
test_utterances = df['utterances']
flattened_list_emotion = [[item] for sublist in test_emotions for item in sublist]
flattened_list_utterances = [item for sublist in test_utterances for item in sublist]

In [41]:
from sklearn.preprocessing import LabelBinarizer
sorted_emotions = sorted(emotions)  #sort the array because Binarizer will automatically do that for one hot encoding
label_binarizer = LabelBinarizer()
label_binarizer.fit(sorted_emotions)
#emotion_numerical_format_test = [emotion_to_id[emotion] for emotion in flattened_list_emotion]
#Binarizer_test = [label_binarizer.transform(emotion) for emotion in flattened_list_emotion]
binarizer_test = []
for emotion in flattened_list_emotion:
    binarized_emotion = label_binarizer.transform(emotion)
    binarizer_test.extend(binarized_emotion.tolist())

In [50]:
binarizer_test[0]
binarizer_test_np = np.array(binarizer_test,dtype=float)

In [51]:
binarizer_test_np

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [13]:
#emotion_numerical_format_test = [emotion_to_id[emotion] for emotion in flattened_list_emotion]

In [ ]:
binarizer_test_np = binarizer_test_np.astype(np.float32).tolist()
binarizer_test_np

In [61]:
from sklearn.model_selection import train_test_split
data_test = {
    'labels' : binarizer_test,#emotions_input_ids, #tokenized emotions
    'utterances': flattened_list_utterances,
}
df_test = pd.DataFrame(data_test)
train_data_test, temp_data_test = train_test_split(df_test, train_size=0.8, shuffle=False)
val_data_test, test_data_test = train_test_split(temp_data_test, test_size=0.5, shuffle=False)

In [62]:
val_data_test

,labels,utterances
28000,"[0, 0, 0, 0, 1, 0, 0]",(noticing a kid who has picked up a copy of
28001,"[0, 0, 0, 1, 0, 0, 0]","Oh yeah, yeah! He's done tons of commercials. ..."
28002,"[0, 0, 0, 1, 0, 0, 0]","Yeah well, he's not gonna get this one."
28003,"[0, 0, 0, 1, 0, 0, 0]",Ben is way cuter than that kid.
28004,"[0, 1, 0, 0, 0, 0, 0]","I mean look at him, look at you,"
...,...,...
31495,"[0, 0, 0, 0, 1, 0, 0]","Just wanna check my horoscope, see if it was r..."
31496,"[0, 0, 0, 0, 0, 0, 1]",Oh my God.
31497,"[0, 0, 0, 0, 1, 0, 0]",Phoebe.
31498,"[0, 0, 0, 0, 1, 0, 0]","Don't look now, but behind us is a guy who has..."


In [ ]:
val_data_test['labels'].tolist()

In [125]:
from datasets import Dataset
def preprocess_function(examples):
    tokenized_inputs = tokenizer(examples['utterances'].tolist(), padding=True, truncation=True)
    # Convert labels to a PyTorch tensor of type long
    labels = examples['labels'].tolist()
    #labels_np = np.array(labels)
    #labels_tensor = torch.tensor(labels_np, dtype=torch.long)

    # Create a PyTorch Dataset
    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels': np.array(labels).astype(np.float32).tolist()#torch.tensor(np.array(labels), dtype=torch.float)
    })
val_dataset_test = preprocess_function(val_data_test)


#validation_hf = Dataset.from_pandas(val_data_test)
#val_dataset_test = validation_hf.map(preprocess_function, batched=True)

In [63]:
import torch
from datasets import Dataset
def preprocess_text(texts):
    tokenized_inputs = tokenizer([" ".join(utterance) for utterance in texts['utterances']],padding='max_length', truncation=True, max_length=max_len_dialogue, return_tensors='pt')

    tokenized_input_ids = []
    tokenized_attention_mask = []
    tokenized_token_type_ids = []

    for utterance_list in texts['utterances']:
      utterance_input_ids = []
      utterance_attention_mask = []
      utterance_token_type_ids = []
      for utterance in utterance_list:
        tokenized_texts = tokenizer(utterance,padding=True,truncation=True,return_tensors='pt')# padding='max_length', max_length=max_len, )
        utterance_input_ids.extend(tokenized_texts.input_ids)
        utterance_attention_mask.extend(tokenized_texts.attention_mask)
        utterance_token_type_ids.extend(tokenized_texts.token_type_ids)

      tokenized_input_ids.append(utterance_input_ids)
      tokenized_attention_mask.append(utterance_attention_mask)
      tokenized_token_type_ids.append(utterance_token_type_ids)
    #labels_tensor = torch.tensor(labels_np[3200:3600,:], dtype=torch.float)
    labels_tensor = torch.tensor(labels_np[3200:3600, :], dtype=torch.float)

    print(labels_tensor.shape)
    print(torch.tensor(tokenized_inputs['input_ids'],dtype=torch.float).shape)
    print(torch.tensor(tokenized_inputs['attention_mask'],dtype=torch.float).shape)
    print(tokenized_inputs['input_ids'][0])
    print(type(tokenized_inputs['input_ids']))

    return Dataset.from_dict({
        'input_ids': tokenized_input_ids,#tokenized_inputs['input_ids'],
        'attention_mask': tokenized_attention_mask,#tokenized_inputs['attention_mask'],
        'labels': labels_tensor
    })

"""def preprocess_data(data, tokenizer, mlb):
    # Tokenize the utterances
    tokenized_inputs = tokenizer(
        [" ".join(utterance) for utterance in data['utterances']],
        padding='max_length',
        truncation=True,
        max_length=128, # we might wanna change this to max_len !
        return_tensors='pt'
    )

    # Encode the emotions
    encoded_labels = mlb.transform(data['emotions'])
    # Convert labels to tensors
    labels_tensor = torch.tensor(encoded_labels, dtype=torch.float)
    # Ensure the encoded labels are the correct shape (this step is crucial to avoid the batch size mismatch)
    if labels_tensor.shape[0] != tokenized_inputs['input_ids'].shape[0]:
        raise ValueError(f"Number of examples {tokenized_inputs['input_ids'].shape[0]} does not match number of labels {labels_tensor.shape[0]}")

    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels': labels_tensor
    })"""

# Now call preprocess_data to process your train and validation data
#train_dataset = preprocess_text(train_data)
val_dataset = preprocess_text(val_data)

"""training = training_hf.map(preprocess_text, batched=True)
validation = validation_hf.map(preprocess_text, batched=True)
test = test_hf.map(preprocess_text, batched=True)"""

NameError: name 'val_data' is not defined

In [64]:
for name, param in model.named_parameters():
    if 'embeddings' in name: # Layer names not containing 'classifier' will be frozen
        param.requires_grad = False

In [127]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test_dir",                 # where to save model
    learning_rate=1e-5,
    per_device_train_batch_size=5,         # accelerate defines distributed training
    per_device_eval_batch_size=50,
    num_train_epochs=3,
    weight_decay=0.1,
    evaluation_strategy="epoch",           # when to report evaluation metrics/losses
    logging_strategy="epoch",
    save_strategy="epoch",                 # when to save checkpoint
    load_best_model_at_end=True,
    report_to='none'                       # disabling wandb (default)
)
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=val_dataset_test,
    eval_dataset=val_dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [78]:
#metrics = compute_metrics(test_prediction_info)
#print(test_prediction_info)
test_prediction_info = trainer.predict(val_dataset)
test_prediction_info.label_ids.shape
test_prediction_info.predictions[0],test_prediction_info.label_ids[0]

(array([ 0.27739397,  0.29370916, -0.1685992 ,  0.3684679 , -0.42971808,
        -0.27069902,  0.37707573, -0.14887775, -0.32769036,  0.07534103,
         0.3421814 ,  0.18997188,  0.51304376,  0.395989  ,  0.90670776,
         0.54127747,  0.71304816,  0.76001567,  0.79307735,  0.6367639 ,
         0.6389566 , -0.04435933,  0.5742843 ,  0.4485132 ], dtype=float32),
 array([0., 0., 0., 0., 0., 3., 2., 0., 4., 7., 7., 7., 7., 7., 7., 7., 7.,
        7., 7., 7., 7., 7., 7., 7.], dtype=float32))

In [55]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader

class EmotionDataset(Dataset):
    def __init__(self, data, labels, tokenizer, max_seq_length=128):
        self.data = data
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        utterances = self.data[idx]  # List of strings, each representing an utterance
        encodings = self.tokenizer(utterances, truncation=True, padding='max_length', max_length=self.max_seq_length, return_tensors='pt')
        print("input_ids shape:", encodings['input_ids'].shape)
        print("attention_mask shape:", encodings['attention_mask'].shape)
        # Make sure 'input_ids' and 'attention_mask' are present in the encoding
        input_ids = encodings['input_ids'].squeeze()
        attention_mask = encodings['attention_mask'].squeeze()

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': label}


# Replace with your actual data
data = padded_dialogues

# Replace with your one-hot encoded labels
labels = numerical_representation

# Hyperparameters
model_name = "bert-base-uncased"
num_emotions = 8
max_seq_length = 128
batch_size = 5
epochs = 1

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

# Create dataset
dataset = EmotionDataset(data, labels, tokenizer, max_seq_length)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./emotion_classification",
    per_device_train_batch_size=batch_size,
    num_train_epochs=epochs,
    logging_dir="./logs",
)

# Initialize the model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_emotions)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=lambda data: {'input_ids': torch.stack([item['input_ids'] for item in data]),
                                'attention_mask': torch.stack([item['attention_mask'] for item in data]),
                                'labels': torch.stack([item['labels'] for item in data])}
)

# Train the model
trainer.train()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

input_ids shape: torch.Size([24, 128])
attention_mask shape: torch.Size([24, 128])
input_ids shape: torch.Size([24, 128])
attention_mask shape: torch.Size([24, 128])
input_ids shape: torch.Size([24, 128])
attention_mask shape: torch.Size([24, 128])
input_ids shape: torch.Size([24, 128])
attention_mask shape: torch.Size([24, 128])
input_ids shape: torch.Size([24, 128])
attention_mask shape: torch.Size([24, 128])


ValueError: too many values to unpack (expected 2)

In [ ]:
#metrics = compute_metrics(test_prediction_info)
#print(test_prediction_info)
test_prediction_info = trainer.predict(val_dataset)
test_prediction_info.label_ids.shape
test_prediction_info.predictions[0],test_prediction_info.label_ids[0]

In [ ]:
val_data['emotions'][3200]

### Nalin's scribbles below

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Apply sigmoid to logits and threshold to get multi-label predictions
    sigmoid_logits = 1 / (1 + np.exp(-logits))  # Sigmoid function
    threshold = 0.5
    predictions = (sigmoid_logits > threshold).astype(int)

    # Calculate F1 score for each individual label/class
    f1_scores = [f1_score(labels[:, i], predictions[:, i], average='binary') for i in range(num_labels)]

    # Sequence F1: compute the F1-score for each dialogue and report the average score
    sequence_f1 = np.mean(f1_scores)

    # Unrolled Sequence F1: flatten all utterances and compute the F1-score
    unrolled_labels = labels.flatten()
    unrolled_predictions = predictions.flatten()
    unrolled_sequence_f1 = f1_score(unrolled_labels, unrolled_predictions, average='binary')

    return {
        'sequence_f1': sequence_f1,
        'unrolled_sequence_f1': unrolled_sequence_f1
    }

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset
import torch

emotions = df['emotions'].explode().unique()
num_labels = len(emotions)
# Initialize the tokenizer
model_card = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_card)

# Initialize MultiLabelBinarizer for emotions
mlb = MultiLabelBinarizer()
mlb.fit(train_data['emotions'])

def preprocess_data(data, tokenizer, mlb):
    # Tokenize the utterances
    tokenized_inputs = tokenizer(
        [" ".join(utterance) for utterance in data['utterances']],
        padding='max_length',
        truncation=True,
        max_length=128, # we might wanna change this to max_len !
        return_tensors='pt'
    )

    # Encode the emotions
    encoded_labels = mlb.transform(data['emotions'])
    # Convert labels to tensors
    labels_tensor = torch.tensor(encoded_labels, dtype=torch.float)
    # Ensure the encoded labels are the correct shape (this step is crucial to avoid the batch size mismatch)
    if labels_tensor.shape[0] != tokenized_inputs['input_ids'].shape[0]:
        raise ValueError(f"Number of examples {tokenized_inputs['input_ids'].shape[0]} does not match number of labels {labels_tensor.shape[0]}")

    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels': labels_tensor
    })

# Now call preprocess_data to process your train and validation data
train_dataset = preprocess_data(train_data, tokenizer, mlb)
val_dataset = preprocess_data(val_data, tokenizer, mlb)

print("Single train example:", train_dataset[0])
print("Single validation example:",val_dataset[0])

In [ ]:
val_dataset['input_ids'][0]

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy = "epoch"
)

# Initialize the model with the number of emotion labels
num_labels = len(mlb.classes_)
model = BertForSequenceClassification.from_pretrained(model_card, num_labels=num_labels)
# Freeze the BERT embedding layer weights
for name, param in model.named_parameters():
    if 'classifier' not in name: # Layer names not containing 'classifier' will be frozen
        param.requires_grad = False

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)
# Train the model
trainer.train()
# Train the model
trainer.train()
results = trainer.evaluate()

print(results)

In [ ]:
test_prediction_info = trainer.predict(val_dataset)
#metrics = compute_metrics(test_prediction_info)
#print(test_prediction_info)
test_prediction_info.predictions[0],test_prediction_info.label_ids[0]

## For two classifier heads, one common model


In [ ]:
!pip install datasets==2.13.2
!pip install accelerate -U

In [ ]:
from transformers import BertPreTrainedModel, BertModel
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.nn import BCEWithLogitsLoss, Linear
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset

class BertForMultiLabelSequenceClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels_emotions, num_labels_triggers):
        super().__init__(config)
        self.num_labels_emotions = num_labels_emotions
        self.num_labels_triggers = num_labels_triggers

        self.bert = BertModel(config)
        self.classifier_emotions = Linear(config.hidden_size, num_labels_emotions)
        self.classifier_triggers = Linear(config.hidden_size, num_labels_triggers)

        # You can initialize weights here if needed

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels_emotions=None,
        labels_triggers=None,
        return_dict=None,
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        pooled_output = outputs[1]

        logits_emotions = self.classifier_emotions(pooled_output)
        logits_triggers = self.classifier_triggers(pooled_output)

        loss = None
        # Calculate loss only if both labels are provided
        if labels_emotions is not None and labels_triggers is not None:
            loss_fct = BCEWithLogitsLoss()
            loss_emotions = loss_fct(logits_emotions.view(-1, self.num_labels_emotions), labels_emotions.view(-1))
            loss_triggers = loss_fct(logits_triggers.view(-1, self.num_labels_triggers), labels_triggers.view(-1))
            loss = loss_emotions + loss_triggers

        # Adjust the return statement to match what your compute_loss method expects
        if not return_dict:
            outputs = (logits_emotions, logits_triggers)
            if loss is not None:
                outputs = (loss,) + outputs
        else:
            outputs = {'loss': loss, 'logits_emotions': logits_emotions, 'logits_triggers': logits_triggers}

        return outputs
        """
        if not return_dict:
            print(f"Logits Emotions: {logits_emotions.shape}, Logits Triggers: {logits_triggers.shape}")
            output = (logits_emotions, logits_triggers) if loss is None else (loss, logits_emotions, logits_triggers)
            return (loss, logits_emotions, logits_triggers) if loss is not None else (logits_emotions, logits_triggers)
        else:
            raise NotImplementedError("return_dict is set to True, but this model does not support it.")
            """

In [ ]:
# Initialize MultiLabelBinarizers
mlb_emotions = MultiLabelBinarizer()
mlb_triggers = MultiLabelBinarizer()
mlb_emotions.fit(train_data['emotions'])
mlb_triggers.fit(train_data['triggers'])

# Initialize the tokenizer
model_card = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_card)

In [ ]:
def preprocess_data(data, tokenizer, mlb_emotions, mlb_triggers):
    tokenized_inputs = tokenizer(
      [" ".join(utterance) for utterance in data['utterances']],
      padding='max_length',
      truncation=True,
      max_length=128,
      return_tensors='pt'
    )

    # Encode the emotions and triggers
    encoded_labels_emotions = mlb_emotions.transform(data['emotions'])
    encoded_labels_triggers = mlb_triggers.transform(data['triggers'])

    # Convert labels to tensors and reshape
    labels_emotions = torch.tensor(encoded_labels_emotions, dtype=torch.float).view(-1, mlb_emotions.classes_.size)
    labels_triggers = torch.tensor(encoded_labels_triggers, dtype=torch.float).view(-1, mlb_triggers.classes_.size)

    return Dataset.from_dict({
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask'],
        'labels_emotions': labels_emotions,
        'labels_triggers': labels_triggers
    })

# Then preprocess the training and validation data
train_dataset = preprocess_data(train_data, tokenizer, mlb_emotions, mlb_triggers)
val_dataset = preprocess_data(val_data, tokenizer, mlb_emotions, mlb_triggers)

print("Single train example:", train_dataset[0])
print("Single validation example:",val_dataset[0])


In [ ]:
from sklearn.metrics import f1_score

# Define your compute_metrics function to handle both sets of labels
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits_emotions = logits[0]
    logits_triggers = logits[1]
    labels_emotions = labels[0]
    labels_triggers = labels[1]
    # Sigmoid function to convert logits to probabilities
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    # Threshold to convert probabilities to binary predictions
    threshold = 0.5

    # Convert logits to probabilities
    probs_emotions = sigmoid(logits_emotions)
    probs_triggers = sigmoid(logits_triggers)

    # Convert probabilities to binary predictions
    preds_emotions = (probs_emotions > threshold).astype(int)
    preds_triggers = (probs_triggers > threshold).astype(int)

    # Calculate the f1-score for emotions
    f1_emotions = f1_score(labels_emotions, preds_emotions, average='macro')
    # Calculate the f1-score for triggers
    f1_triggers = f1_score(labels_triggers, preds_triggers, average='macro')

    # To calculate sequence-level metrics, we might need to modify data processing

    # to include sequence IDs, and then group by sequence ID before calculating metrics.
    return {
        'f1_emotions': f1_emotions,
        'f1_triggers': f1_triggers,
    }

In [ ]:
#TODO make this compatible with model class by changing the way the absence of loss is handled, similar to CustomTrainer4

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Unpack inputs
        labels_emotions = inputs.pop("labels_emotions", None)
        labels_triggers = inputs.pop("labels_triggers", None)

        # Forward pass
        outputs = model(**inputs)

        # If loss is not included in outputs, calculate it using the logits
        if len(outputs) == 2:
            logits_emotions, logits_triggers = outputs
            loss_fct = BCEWithLogitsLoss()
            if labels_emotions is not None and labels_triggers is not None:
                loss_emotions = loss_fct(logits_emotions.view(-1, self.model.num_labels_emotions), labels_emotions.view(-1, self.model.num_labels_emotions))
                loss_triggers = loss_fct(logits_triggers.view(-1, self.model.num_labels_triggers), labels_triggers.view(-1, self.model.num_labels_triggers))
                loss = loss_emotions + loss_triggers
        elif len(outputs) == 3:
            # Outputs include a loss
            loss, logits_emotions, logits_triggers = outputs
        else:
            raise ValueError(f"Unexpected number of outputs from model: {len(outputs)}")

        # If loss was not computed within the model, we compute it based on the logits and the labels
        if loss is None and labels_emotions is not None and labels_triggers is not None:
            loss_fct = BCEWithLogitsLoss()
            loss_emotions = loss_fct(logits_emotions.view(-1, self.model.num_labels_emotions), labels_emotions.view(-1))
            loss_triggers = loss_fct(logits_triggers.view(-1, self.model.num_labels_triggers), labels_triggers.view(-1))
            loss = loss_emotions + loss_triggers

        return (loss, outputs) if return_outputs else loss


In [ ]:
class CustomTrainer4(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Unpack inputs
        labels_emotions = inputs.pop("labels_emotions", None)
        labels_triggers = inputs.pop("labels_triggers", None)

        # Forward pass
        outputs = model(**inputs)

        # If loss is not included in outputs, calculate it using the logits
        if 'loss' not in outputs:
            logits_emotions = outputs['logits_emotions']
            logits_triggers = outputs['logits_triggers']
            loss_fct = BCEWithLogitsLoss()
            if labels_emotions is not None:
                loss_emotions = loss_fct(logits_emotions.view(-1, self.model.num_labels_emotions), labels_emotions.view(-1))
            if labels_triggers is not None:
                loss_triggers = loss_fct(logits_triggers.view(-1, self.model.num_labels_triggers), labels_triggers.view(-1))
            loss = loss_emotions + loss_triggers
            outputs['loss'] = loss
        else:
            # Loss is included in outputs
            loss = outputs['loss']

        return (loss, outputs) if return_outputs else loss


In [ ]:
# Initialize model with the number of emotion and trigger labels
num_labels_emotions = len(mlb_emotions.classes_)
num_labels_triggers = len(mlb_triggers.classes_)
model = BertForMultiLabelSequenceClassification.from_pretrained(
    model_card,
    num_labels_emotions=num_labels_emotions,
    num_labels_triggers=num_labels_triggers
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Assuming train_dataset is a Hugging Face dataset with the correct columns
example = train_dataset[0]

# Now, before passing tensors to the model, ensure they are also on the same device
example_tensors = {k: torch.tensor([v]).to(device) for k, v in example.items()}

# Call the model
with torch.no_grad():
    outputs = model(**example_tensors)
    print(outputs)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy = "epoch"
    report_to = ""
)

# # Initialize the model with the number of emotion labels
# num_labels = len(mlb.classes_)
# model = BertForSequenceClassification.from_pretrained(model_card,
#                                                       num_labels=num_labels,
#                                                        )

# Freeze the BERT embedding layer weights
for name, param in model.named_parameters():
    if 'classifier' not in name: # Layer names not containing 'classifier' will be frozen
        param.requires_grad = False

# Initialize the Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics
)

# Train the model
trainer.train()
results = trainer.evaluate()

print(results)
